## Exploring UZF

This worked example sets up a MODFLOW 6 simulation with multiple GWF models to explore the sensitivity of various UZF parameters (e.g., Brooks-Corey epsilon, vks, etc.) within the UZF package and what impact it can have on the flow solution

In [ ]:
# Imports
import os
import sys

# Suppress warning messages
import warnings
from pathlib import Path

import flopy
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import HTML
from matplotlib.collections import LineCollection

warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", DeprecationWarning)
warnings.simplefilter("ignore", FutureWarning)

### Set simulation name and workspace

In [ ]:
sim_name = "unsat-comp"
sim_ws = Path("./temp/gwf-uzf")

### Set some model input values

In [ ]:
# Model units
length_units = "meters"
time_units = "seconds"

# Solver parameters
nouter, ninner = 100, 300
hclose, rclose, relax = 1e-8, 1e-8, 0.97

# Set some parameter values
nper = 365  # Number of periods
nstp = 1  # Number of time steps
perlen = 86400  # Simulation time length ($s$)
nlay = 120  # Number of layers
nrow = 1  # Number of rows
ncol = 1  # Number of columns
system_length = 60.0  # Length of system ($m$)
delr = 1.0  # Column width ($m$)
delc = 1.0  # Row width ($m$)
delv_str = "ranges from 0.1 to 1"  # Layer thickness
top = 60.0  # Top of the model ($m$)
keff = 3e-6
hydraulic_conductivity = keff  # Hydraulic conductivity ($m s^{-1}$)
alphal = 0.0  # Longitudinal dispersivity ($m$)
alphat = 0.0  # Transverse dispersivity ($m$)
T_az = 5  # Ambient temperature ($^o C$)
dT = 5  # Temperature variation ($^o C$)
cpw = 4174.0
cps = 1600.0
rhow = 1000.0
rhos = 2630.0
lambda_w = 0.58
lambda_s = 2
bulk_dens = rhos  # Bulk density ($kg/m^3$)
kd = cps / (cpw * rhow)  # Distribution coefficient (unitless)
ktw = lambda_w
lhv = 0
Zw = 55
Sw = 1
u_gradient = 0.3 / 60
darcy_flux = keff * (u_gradient)
lambda_s1 = 2.4
lambda_s2 = lambda_s1
thtr = 0.01
n1 = 0.24
n2 = 0.14

In [ ]:
# Stress period input
per_data = []
for k in range(nper):
    per_data.append((perlen, nstp, 1.0))
per_mf6 = per_data

# Geometry input
tp = top
botm = []
for i in range(nlay):
    if i == 0:
        botm.append(59.9)
    elif i == 119:
        botm.append(0.0)
    else:
        botm.append(60 - i * 0.5)

### Setting some boundary package data

In [ ]:
# Head input
chd_data = {}
chd_data[0] = [[(119, 0, 0), 53.875]]

chd_mf6 = chd_data

# Recharge input
wel_data = {}
wel_data[0] = [[(0, 0, 0), "prcp"]]
wel_mf6 = wel_data

# Initial temperature input
strt_conc = T_az * np.ones((nlay, 1, 1), dtype=np.float32)

### Read in time series input and prepare it for MODFLOW 6 time series utility

In [ ]:
# Boundary temperature input
data_path = "../../data/uzf"
df = pd.read_csv(os.path.join(data_path, "Daily_dat.csv"), index_col=0)
df_2 = pd.read_csv(os.path.join(data_path, "Daily_dat_2.csv"), index_col=0)
ctp_spd = {}
for i in range(nper):
    ctp_spd[i] = [
        [(0, 0, 0), df["TAVG_rmFrz"][i]]
    ]  # ctp_spd.update({i: [[(0, 0, 0), df['TAVG_rmFrz'][i]]]})

prcp_ts_data = []
pet_ts_data = []
temp_ts_data = []

for n in range(0, len(df)):
    time = n * 86400
    prcp = df["PRCP"][n] / 1000 / 86400
    pet = df["PET"][n] / 1000 / 86400
    temp = df["TAVG_rmFrz"][n]
    prcp_ts_data.append((time, prcp))
    temp_ts_data.append((time, temp))
    pet_ts_data.append((time, pet))

prcp_ts_dict_base = {
    "filename": "prcp.ts",
    "time_series_namerecord": "prcp",
    "timeseries": prcp_ts_data,
    "interpolation_methodrecord": "linearend",
}

# Make another copy of the time series file since two GWF model's cannot use the same input file (a file unit number issue)
prcp_ts_dict_eps = {
    "filename": "prcp_eps.ts",
    "time_series_namerecord": "prcp_eps",
    "timeseries": prcp_ts_data.copy(),
    "interpolation_methodrecord": "linearend",
}

prcp_ts_dict_vks = {
    "filename": "prcp_vks.ts",
    "time_series_namerecord": "prcp_vks",
    "timeseries": prcp_ts_data.copy(),
    "interpolation_methodrecord": "linearend",
}

prcp_ts_dict_uzet = {
    "filename": "prcp_uzet.ts",
    "time_series_namerecord": "prcp_uzet",
    "timeseries": prcp_ts_data.copy(),
    "interpolation_methodrecord": "linearend",
}

prcp_ts_dict_theta_s = {
    "filename": "prcp_theta_s.ts",
    "time_series_namerecord": "prcp_theta_s",
    "timeseries": prcp_ts_data.copy(),
    "interpolation_methodrecord": "linearend",
}

pet_ts_dict = {
    "filename": "pet.ts",
    "time_series_namerecord": "pet",
    "timeseries": pet_ts_data,
    "interpolation_methodrecord": "linearend",
}

### Setup objects for UZF 

Includes two functions to help create the UZF input later on

In [ ]:
# transient uzf info
def set_uzf_spd_info(gwfname, ietflg, surfdep):
    # The following is needed because different models within the same simulation cannot all access the same file.
    if "base" in gwfname:
        finf = "prcp"
    elif "eps" in gwfname:
        finf = "prcp_eps"
    elif "vks" in gwfname:
        finf = "prcp_vks"
    elif "uzet" in gwfname:
        finf = "prcp_uzet"
    elif "theta" in gwfname:
        finf = "prcp_theta_s"

    zero = 0.0
    extwc = 0.0
    if ietflg == 0:
        pet = 0.0
        extdp = 0.0
        uzf_spd = [[0, finf, pet, extdp, extwc, zero, zero, zero]]
    elif ietflg != 0:
        pet = "pet"
        extdp = 3.9  # meters
        uzf_spd = []
        for i in np.arange(nlay):
            if (top - surfdep) - botm[i] < extdp:
                if i > 0:
                    finf = 0.0
                uzf_spd.append([i, finf, pet, extdp, extwc, zero, zero, zero])

    uzf_spd = {0: uzf_spd}

    return uzf_spd


def add_uzf_obs(uz_obs_loc, obsnm, iuz, elev):
    uz_obs_loc.append((obsnm, "water-content", iuz, elev))
    return uz_obs_loc


def add_uzf_wc_profile_obs():
    uz_obs_loc = []
    iuz = 1
    depth = round((top - botm[0]) / 2, 2)
    elev = top - depth
    # Top uzf object (has a unique thickness)
    obsnm = "uzf" + str(iuz).zfill(2) + "_elev=" + str(elev)
    uz_obs_loc = add_uzf_obs(uz_obs_loc, obsnm, iuz, depth)

    # Loop through upper X-number cells (17 cells currently)
    increment = 0.1
    for iuz in np.arange(1, 17, 1):
        depth1 = depth  # 0.05
        depth2 = round(depth1 + increment, 2)  # 0.15
        depth3 = round(depth2 + increment, 2)  # 0.25
        depth4 = round(depth3 + increment, 2)  # 0.35
        depth5 = round(depth4 + increment, 2)  # 0.45
        elev1 = round(botm[iuz - 1] - depth1, 2)
        elev2 = round(botm[iuz - 1] - depth2, 2)
        elev3 = round(botm[iuz - 1] - depth3, 2)
        elev4 = round(botm[iuz - 1] - depth4, 2)
        elev5 = round(botm[iuz - 1] - depth5, 2)
        obsnm1 = "uzf" + str(iuz).zfill(2) + "_elev=" + str(elev1)
        obsnm2 = "uzf" + str(iuz).zfill(2) + "_elev=" + str(elev2)
        obsnm3 = "uzf" + str(iuz).zfill(2) + "_elev=" + str(elev3)
        obsnm4 = "uzf" + str(iuz).zfill(2) + "_elev=" + str(elev4)
        obsnm5 = "uzf" + str(iuz).zfill(2) + "_elev=" + str(elev5)
        uz_obs_loc = add_uzf_obs(uz_obs_loc, obsnm1, iuz + 1, depth1)
        uz_obs_loc = add_uzf_obs(uz_obs_loc, obsnm2, iuz + 1, depth2)
        uz_obs_loc = add_uzf_obs(uz_obs_loc, obsnm3, iuz + 1, depth3)
        uz_obs_loc = add_uzf_obs(uz_obs_loc, obsnm4, iuz + 1, depth4)
        if iuz > 1:
            uz_obs_loc = add_uzf_obs(uz_obs_loc, obsnm5, iuz + 1, depth5)

    return uz_obs_loc

### Function for creating a GWF model

Since multiple GWF models will be added to the MODFLOW 6 simulation, define a function to do the work multiple times.

**Note**: The function call accepts arguments that define parameter values for the UZF package, for example the Brooks-Corey epsilon ($\epsilon$). This allows different calls to the this function to instantiate the UZF package with different parameter specifications for exploring their impact on flow in the unsaturated zone.

In [ ]:
def add_gwf_model(sim, gwfname, eps=4, keff=3e-6, ietflg=0):
    msg0 = "EPSILON must be between 3.5 and 14.0"
    assert eps > 3.5 and eps < 15.0, msg0

    # Instantiating MODFLOW 6 groundwater flow model
    gwf = flopy.mf6.ModflowGwf(
        sim,
        modelname=gwfname,
        save_flows=True,
        newtonoptions=True,
        model_nam_file="{}.nam".format(gwfname),
    )

    # Instantiating MODFLOW 6 solver for flow model
    imsgwf = flopy.mf6.ModflowIms(
        sim,
        print_option="SUMMARY",
        outer_dvclose=hclose,
        outer_maximum=nouter,
        under_relaxation="NONE",
        inner_maximum=ninner,
        inner_dvclose=hclose,
        rcloserecord=rclose,
        linear_acceleration="BICGSTAB",
        scaling_method="NONE",
        reordering_method="NONE",
        relaxation_factor=relax,
        filename="{}.ims".format(gwfname),
    )
    sim.register_ims_package(imsgwf, [gwfname])

    # Instantiating MODFLOW 6 discretization package
    flopy.mf6.ModflowGwfdis(
        gwf,
        length_units=length_units,
        nlay=nlay,
        nrow=nrow,
        ncol=ncol,
        delr=delr,
        delc=delc,
        top=top,
        botm=botm,
        filename="{}.dis".format(gwfname),
    )

    # Instantiating MODFLOW 6 node-property flow package
    flopy.mf6.ModflowGwfnpf(
        gwf,
        save_specific_discharge=True,
        icelltype=1,
        k=hydraulic_conductivity,
        filename="{}.npf".format(gwfname),
    )

    # Instantiate MODFLOW 6 storage package
    if "theta" in gwfname:
        n_use = n2
    else:
        n_use = n1

    flopy.mf6.ModflowGwfsto(
        gwf,
        ss=1e-6,
        sy=n_use,
        iconvert=1,
        # steady_state={0: True},
        filename="{}.sto".format(gwfname),
    )

    # Instantiating MODFLOW 6 initial conditions package for flow model
    flopy.mf6.ModflowGwfic(gwf, strt=Zw, filename="{}.ic".format(gwfname))

    # Instantiating MODFLOW 6 constant head package
    flopy.mf6.ModflowGwfchd(
        gwf,
        stress_period_data=chd_mf6,
        filename="{}.chd".format(gwfname),
    )

    # Unsaturated-zone flow package
    uz_obs_loc = add_uzf_wc_profile_obs()
    uzf_obs = {f"{gwfname}.uzfobs": uz_obs_loc}
    if "base" in gwfname:
        prcp_ts_dict_entry = prcp_ts_dict_base
    elif "eps" in gwfname:
        prcp_ts_dict_entry = prcp_ts_dict_eps
    elif "vks" in gwfname:
        prcp_ts_dict_entry = prcp_ts_dict_vks
    elif "uzet" in gwfname:
        prcp_ts_dict_entry = prcp_ts_dict_uzet
    elif "theta" in gwfname:
        prcp_ts_dict_entry = prcp_ts_dict_theta_s

    # The following is for the upper-most (land surface) cell
    # iuzno  cellid landflg ivertcn surfdp vks thtr thts thti eps [bndnm]
    surfdep = 0.00001
    uzf_pkdat = [
        [0, (0, 0, 0), 1, 1, surfdep, keff, thtr, thtr + n1, 0.025, eps]
    ]

    # Continue building the UZF stack of objects
    for iuzno in np.arange(1, nlay, 1):
        if iuzno < nlay - 1:
            ivertconn = iuzno + 1
        else:
            ivertconn = -1
        # Add the UZF object to the packagedata list object
        uzf_pkdat.append(
            [
                iuzno,
                (iuzno, 0, 0),
                0,
                ivertconn,
                0.0,
                keff,
                thtr,
                thtr + n_use,
                0.025,
                eps,
            ]
        )

    # Call function to return stress period ('PERIODDATA' block) information
    uzf_spd = set_uzf_spd_info(gwfname, ietflg, surfdep)
    if ietflg == 0:
        simulate_et = False
        unsat_etwc = False
        linear_gwet = False
    else:
        simulate_et = True
        unsat_etwc = True
        linear_gwet = True

    uzf = flopy.mf6.ModflowGwfuzf(
        gwf,
        print_flows=True,
        save_flows=True,
        wc_filerecord=gwfname + ".uzfwc.bin",
        timeseries=prcp_ts_dict_entry,
        simulate_et=simulate_et,
        unsat_etwc=unsat_etwc,
        simulate_gwseep=False,
        linear_gwet=linear_gwet,
        boundnames=False,
        observations=uzf_obs,
        ntrailwaves=15,
        nwavesets=500,
        nuzfcells=len(uzf_pkdat),
        packagedata=uzf_pkdat,
        perioddata=uzf_spd,
        budget_filerecord=f"{gwfname}.uzf.bud",
        pname="UZF-1",
        filename=f"{gwfname}.uzf",
    )
    if "uzet" in gwfname:
        uzf.ts.append_package(
            filename="pet.ts",
            time_series_namerecord="pet",
            timeseries=pet_ts_data,
            interpolation_methodrecord="linearend",
        )

    # Instantiating MODFLOW 6 output control package for flow model
    flopy.mf6.ModflowGwfoc(
        gwf,
        head_filerecord="{}.hds".format(gwfname),
        budget_filerecord="{}.cbc".format(gwfname),
        headprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
        saverecord=[("HEAD", "LAST"), ("BUDGET", "LAST")],
        printrecord=[("HEAD", "LAST"), ("BUDGET", "LAST")],
    )

    return sim

### Create the simulation and add the models

In [ ]:
sim = flopy.mf6.MFSimulation(
    sim_name=sim_name, sim_ws=sim_ws, exe_name="mf6", version="mf6"
)

# Instantiating MODFLOW 6 time discretization
flopy.mf6.ModflowTdis(
    sim, nper=nper, perioddata=per_mf6, time_units=time_units
)

# Add the first GWF model: "base case"
gwfnamebase = "gwf-base"
sim = add_gwf_model(sim, gwfnamebase, eps=3.6, keff=3e-6, ietflg=0)

# Add a second GWF model: adjust Brooks-Corey epsilon
gwfnamescen1 = "gwf-eps"
sim = add_gwf_model(sim, gwfnamescen1, eps=7.2, keff=3e-6, ietflg=0)

# Add a third GWF model: restore Brooks-Corey epsilon, adjust vertical hydraulic conductivity
gwfnamescen2 = "gwf-vks"
sim = add_gwf_model(
    sim, gwfnamescen2, eps=3.6, keff=3e-5, ietflg=0
)  # More like a clean sand

# Add a fourth GWF model: params same as "base case" scenario but with ET
gwfnamescen3 = "gwf-uzet"
sim = add_gwf_model(sim, gwfnamescen3, ietflg=1)  # More like a clean sand

# Add a fifth GWF model: explore effect of saturated water content
gwfnamescen4 = "gwf-theta_s"
sim = add_gwf_model(
    sim,
    gwfnamescen4,
    ietflg=0,
)  # More like a clean sand

sim.write_simulation(silent=False)

success, buff = sim.run_simulation(silent=False, report=True)
assert success, buff

### Load and animate UZF & UZE output

In [ ]:
# load temperature, model 1
gwf1 = sim.get_model(gwfnamebase)
gwf2 = sim.get_model(gwfnamescen1)
gwf3 = sim.get_model(gwfnamescen2)
gwf4 = sim.get_model(gwfnamescen3)
gwf5 = sim.get_model(gwfnamescen4)

# Water content output
pth = sim.sim_path
flb = gwfnamebase + ".uzfobs"
uzdat_base = pd.read_csv(os.path.join(pth, flb))

fleps = gwfnamescen1 + ".uzfobs"
uzdat_eps = pd.read_csv(os.path.join(pth, fleps))

flvks = gwfnamescen2 + ".uzfobs"
uzdat_vks = pd.read_csv(os.path.join(pth, flvks))

fluzet = gwfnamescen3 + ".uzfobs"
uzdat_uzet = pd.read_csv(os.path.join(pth, fluzet))

fltheta = gwfnamescen4 + ".uzfobs"
uzdat_theta = pd.read_csv(os.path.join(pth, fltheta))

# Timing information for printing on animation
uzdat_base["time"] = uzdat_base["time"] / 86400
uzdat_base["time"] = uzdat_base["time"].astype(int)
uzdat_base.set_index("time", inplace=True)

uzdat_eps["time"] = uzdat_eps["time"] / 86400
uzdat_eps["time"] = uzdat_eps["time"].astype(int)
uzdat_eps.set_index("time", inplace=True)

uzdat_vks["time"] = uzdat_vks["time"] / 86400
uzdat_vks["time"] = uzdat_vks["time"].astype(int)
uzdat_vks.set_index("time", inplace=True)

uzdat_uzet["time"] = uzdat_uzet["time"] / 86400
uzdat_uzet["time"] = uzdat_uzet["time"].astype(int)
uzdat_uzet.set_index("time", inplace=True)

uzdat_theta["time"] = uzdat_theta["time"] / 86400
uzdat_theta["time"] = uzdat_theta["time"].astype(int)
uzdat_theta.set_index("time", inplace=True)

# Mine all observations from obs names
depths_base = uzdat_base.columns
depths_base = [
    round(float(d.strip().split("=")[-1]), 2) for d in depths_base if "=" in d
]

depths_eps = uzdat_eps.columns
depths_eps = [
    round(float(d.strip().split("=")[-1]), 2) for d in depths_eps if "=" in d
]

depths_vks = uzdat_vks.columns
depths_vks = [
    round(float(d.strip().split("=")[-1]), 2) for d in depths_vks if "=" in d
]

depths_uzet = uzdat_uzet.columns
depths_uzet = [
    round(float(d.strip().split("=")[-1]), 2) for d in depths_uzet if "=" in d
]

depths_theta = uzdat_theta.columns
depths_theta = [
    round(float(d.strip().split("=")[-1]), 2) for d in depths_theta if "=" in d
]

norm = plt.Normalize(0.5, 20.0)

layelvs = [top] + botm
cell_half_thk = [
    round((up - down) / 2, 2) for (up, down) in zip(layelvs[:-1], layelvs[1:])
]
node_elvs = np.array(layelvs[:-1]) - np.array(cell_half_thk)
y = layelvs[0:20]
x = [0.5 for i in np.arange(len(y))]

In [ ]:
# Animate results

matplotlib.rcParams["animation.embed_limit"] = 2**128

fig, (ax1, ax2, ax3, ax4) = plt.subplots(
    nrows=1,
    ncols=4,
    figsize=(8.5, 4),
    sharey=True,
    # gridspec_kw=dict(width_ratios=[3, 1, 1], wspace=0.2)
)

# Look at the effects of Brooks-Corey epsilon
ax1.set_ylabel("Depth (m)")
ax1.set_xlabel("Water Content")
(l1,) = ax1.plot([], [], "b-", mfc="none", label=r"$\theta; \epsilon = 3.6$")
(l2,) = ax1.plot([], [], "r-", mfc="none", label=r"$\theta; \epsilon = 7.2$")
line_eps = [l1, l2]
ax1.legend(loc="lower right")
ax1.set_xlim(0.0, 0.25)
ax1.set_ylim(52.0, 60.1)
for yline in layelvs[: len(y)]:
    ax1.axhline(yline, color="gray", linewidth=0.5)


# Look at the effects of vertical hydraulic conductivity
ax2.set_xlabel("Water Content")
(l3,) = ax2.plot(
    [], [], "b-", mfc="none", label=r"$\theta; vks = 3e-6 \dfrac{m}{s}$"
)
(l4,) = ax2.plot(
    [], [], "r-", mfc="none", label=r"$\theta; vks = 3e-5 \dfrac{m}{s}$"
)
line_vks = [l3, l4]
ax2.legend(loc="lower right")
ax2.set_xlim(0.0, 0.25)
ax2.set_ylim(52.0, 60.1)
for yline in layelvs[: len(y)]:
    ax2.axhline(yline, color="gray", linewidth=0.5)


# Look at the effects of ET from the unsaturated zone
ax3.set_xlabel("Water Content")
(l5,) = ax3.plot([], [], "b-", mfc="none", label=r"$\theta; No ET$")
(l6,) = ax3.plot([], [], "r-", mfc="none", label=r"$\theta; With ET$")
line_uzet = [l5, l6]
ax3.legend(loc="lower right")
ax3.set_xlim(0.0, 0.25)
ax3.set_ylim(52.0, 60.1)
for yline in layelvs[: len(y)]:
    ax3.axhline(yline, color="gray", linewidth=0.5)


# Look at the effects of vertical hydraulic conductivity
ax4.set_xlabel("Water Content")
(l7,) = ax4.plot([], [], "b-", mfc="none", label=r"$\theta; \theta_s = 0.25$")
(l8,) = ax4.plot([], [], "r-", mfc="none", label=r"$\theta; \theta_s = 0.15$")
line_theta = [l7, l8]
ax4.legend(loc="lower right")
ax4.set_xlim(0.0, 0.25)
ax4.set_ylim(52.0, 60.1)
for yline in layelvs[: len(y)]:
    ax4.axhline(yline, color="gray", linewidth=0.5)


times = uzdat_base.index


def init():
    ax1.set_title("Time = %.2f years" % (times[0]))


def update(j):
    # Moisture Content
    XBase = uzdat_base.iloc[j].values
    XScen1 = uzdat_eps.iloc[j].values
    XScen2 = uzdat_vks.iloc[j].values
    XScen3 = uzdat_uzet.iloc[j].values
    XScen4 = uzdat_theta.iloc[j].values
    t = times[j] / 365
    line_eps[0].set_data(XBase, depths_base)
    line_eps[1].set_data(XScen1, depths_eps)
    ax1.set_title("Time = %.2f years" % (t))

    line_vks[0].set_data(XBase, depths_base)
    line_vks[1].set_data(XScen2, depths_vks)
    ax2.set_title("Time = %.2f years" % (t))

    line_uzet[0].set_data(XBase, depths_base)
    line_uzet[1].set_data(XScen3, depths_uzet)
    ax3.set_title("Time = %.2f years" % (t))

    line_theta[0].set_data(XBase, depths_base)
    line_theta[1].set_data(XScen4, depths_theta)
    ax4.set_title("Time = %.2f years" % (t))

    return


ani = animation.FuncAnimation(
    fig, update, interval=100, frames=365
)  # frames=len(times)
HTML(ani.to_jshtml())

In [ ]:
# ani.save("UZF6.gif", writer='pillow',fps=60)